# A simple neural network framework

In this section, we will build a simple neural network framework and use it to re-implement multiclass logistic regression in the previous section. This framework is also the start point of more advanced models in following sections.

## Multiclass logistic regression

In [ ]:
import numpy as np
from narray import set_library

set_library('numpy')

%matplotlib inline

In [ ]:
from dataset import MNIST

train_data = MNIST('data/mnist/train-images-idx3-ubyte.gz', 'data/mnist/train-labels-idx1-ubyte.gz')
test_data = MNIST('data/mnist/t10k-images-idx3-ubyte.gz', 'data/mnist/t10k-labels-idx1-ubyte.gz')

In [ ]:
train_data.show_image(0)

In [ ]:
num_examples = train_data.sample_size
num_inputs = np.prod(train_data.input_dim)
num_outputs = np.prod(train_data.target_dim)

In [ ]:
import nn
import neurons

data_layer = nn.BatchDataLayer(train_data.inputs, train_data.targets, batch_size=64, shuffle=True)
fc_layer = nn.FullyConnectedLayer(num_outputs)
softmax_layer = nn.SoftmaxLayer()
loss_layer = nn.CrossEntropyLayer()
logistic_net = nn.NeuralNetwork(data_layer, [fc_layer, softmax_layer], loss_layer)

In [ ]:
from timeit import default_timer as timer

epochs = 10
learning_rate = .002

for e in range(epochs):
    start = timer()
    loss = logistic_net.train(learning_rate)
    end = timer()
    # print("param", softmax_layer.parameters())
    # print("grad", softmax_layer.gradient())
    print("loss %f (%.3f seconds elapsed)" % (loss, end - start))

In [ ]:
def model_predict(net, x):
    yhat = net.evaluate(x)
    pred = np.argmax(yhat, axis=1)
    return pred

In [ ]:
for i in range(5):
    pred_label = model_predict(logistic_net, test_data.inputs[np.newaxis, i])
    test_data.show_image(i, "prediction %d vs target %d" % (pred_label, test_data.labels[i]))

In [ ]:
pred_labels = model_predict(logistic_net, test_data.inputs)
num_correct = np.sum(pred_labels == test_data.labels)
print(num_correct / test_data.sample_size)

## Multilayer perceptrons

In [ ]:
num_hidden = 256
scale_weight = .01

data_layer = nn.BatchDataLayer(train_data.inputs, train_data.targets, batch_size=64, shuffle=True)
hidden_layer_1 = nn.FullyConnectedLayer(num_hidden, scale_weight, neurons.ReLU)
dropout_layer_1 = nn.DropoutLayer()
hidden_layer_2 = nn.FullyConnectedLayer(num_hidden, scale_weight, neurons.ReLU)
dropout_layer_2 = nn.DropoutLayer()
output_layer = nn.FullyConnectedLayer(num_outputs, scale_weight)
softmax_layer = nn.SoftmaxLayer()
# softmax_layer = nn.FullyConnectedLayer(num_hidden, num_outputs, scale_weight, neurons.Softmax)
loss_layer = nn.CrossEntropyLayer()
mlp_net = nn.NeuralNetwork(data_layer, [hidden_layer_1, dropout_layer_1, hidden_layer_2, dropout_layer_2, output_layer, softmax_layer], loss_layer)

In [ ]:
epochs = 10
learning_rate = .002

for e in range(epochs):
    start = timer()
    loss = mlp_net.train(learning_rate)
    end = timer()
    # print("param", softmax_layer.parameters())
    # print("grad", softmax_layer.gradient())
    print("loss %f (%.3f seconds elapsed)" % (loss, end - start))

In [ ]:
pred_labels = model_predict(mlp_net, test_data.inputs)
num_correct = np.sum(pred_labels == test_data.labels)
print(num_correct / test_data.sample_size)